In [1]:
from huggingface_hub import login
login()

In [2]:
# @title Mount Google Drive

from google.colab import drive

try:
    drive.mount('/content/drive')
    print(" Google Drive mounted successfully!")
    print(" You can now access files at /content/drive/MyDrive/")
except Exception as e:
    print(f" Could not mount Google Drive: {e}")

Mounted at /content/drive
 Google Drive mounted successfully!
 You can now access files at /content/drive/MyDrive/


In [3]:
!pip install flash-attn --no-build-isolation -U bitsandbytes transformers datasets peft trl accelerate packaging ninja sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 63.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached ninja-1.13.0-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (5.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 42.8 MB/s eta 0:00:00
Using cached ninja-1.13.0-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (180 kB)
  Created wheel for flash-attn: filename=flash_attn-2.8.3-cp312-cp312-linux_x86_64.whl size=256040057 sha256=f25da18657a87fc83dc1bfb8b7751b82246e9db355510226b674fd437c34b5fb
  Stored in directory: /root/.cache/pip/wheels/3d/59/46/f282c12c73dd4bb3c2e3fe199f1a0d0f8cec06df0cccfeee27
Successfully built flash-attn


In [4]:
#@title Load trained model
import gradio as gr
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import os

def load_model_safely():
    """Load your trained model"""
    try:
        print("Loading trained mental health model...")
        torch.cuda.empty_cache()

        base_model_path = "meta-llama/Llama-2-7b-chat-hf"
        lora_path = "/content/drive/MyDrive/research/mindspace/final_model"

        # Check if model exists
        if not os.path.exists(lora_path):
            print(f"Model path not found: {lora_path}")
            return None, None, False

        # Load base model
        base_model = AutoModelForCausalLM.from_pretrained(
            base_model_path,
            torch_dtype=torch.float16,
            device_map="auto",
            trust_remote_code=True
        )

        # Load tokenizer
        tokenizer = AutoTokenizer.from_pretrained(lora_path, trust_remote_code=True)
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token

        # Load LoRA adapter
        model = PeftModel.from_pretrained(base_model, lora_path)

        print("Model loaded successfully!")
        return model, tokenizer, True

    except Exception as e:
        print(f"Failed to load model: {e}")
        return None, None, False

# Load model
model, tokenizer, model_loaded = load_model_safely()

Loading trained mental health model...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

Model loaded successfully!


In [5]:
# @title Prompt for conversation

import re

def clean_therapeutic_response(response):
    """Clean response specifically for therapeutic conversations"""
    # Remove action descriptions in asterisks
    response = re.sub(r'\*[^*]+\*', '', response)

    # Remove common unwanted patterns
    unwanted_patterns = [
        "Context:", "Response:", "Client:", "Therapist:", "Human:", "Assistant:",
        "[INST]", "[/INST]", "<s>", "</s>", "<<SYS>>", "<</SYS>>",
        "*nods*", "*smiles*", "*leans forward*", "*gestures*"
    ]

    for pattern in unwanted_patterns:
        response = response.replace(pattern, "")

    # Clean up whitespace
    response = re.sub(r'\s+', ' ', response).strip()

    return response

def ensure_complete_sentence(response):
    """Ensure response ends with complete sentence"""
    response = response.strip()

    # If response doesn't end with sentence-ending punctuation
    if not response.endswith(('.', '!', '?')):
        # Find the last complete sentence
        last_period = response.rfind('.')
        last_exclamation = response.rfind('!')
        last_question = response.rfind('?')

        last_punctuation = max(last_period, last_exclamation, last_question)

        if last_punctuation > 0:
            # Truncate to last complete sentence
            response = response[:last_punctuation + 1]
        else:
            # If no sentence-ending punctuation found, add period
            response = response.strip() + "."

    return response.strip()

def therapy_response(message, history):
    """Generate response - trained model first, minimal fallback"""

    if not message.strip():
        return "I'm here to listen. What would you like to share with me today?"

    # Try trained model first
    if model_loaded and model is not None:
        try:
            # Use your context-response training format
            prompt = f"""<s>[INST] <<SYS>>
You are a professional AI therapist. Provide empathetic, supportive therapeutic responses based on your training.
<</SYS>>

Context: {message.strip()}

Response: [/INST]"""

            # Tokenize
            inputs = tokenizer(
                prompt,
                return_tensors="pt",
                max_length=512,
                truncation=True,
                padding=True
            )

            if torch.cuda.is_available():
                inputs = {k: v.to("cuda") for k, v in inputs.items()}

            # Generate
            with torch.no_grad():
                outputs = model.generate(
                    **inputs,
                    max_new_tokens=200,
                    temperature=0.7,
                    do_sample=True,
                    top_p=0.9,
                    pad_token_id=tokenizer.eos_token_id,
                    eos_token_id=tokenizer.eos_token_id
                )

            # Decode
            response = tokenizer.decode(outputs[0], skip_special_tokens=True)

            # Extract response
            response_start = response.find("[/INST]")
            if response_start != -1:
                response = response[response_start + 7:].strip()

            # Clean response using the enhanced function
            response = clean_therapeutic_response(response)

            # ADDED: Ensure complete sentence
            response = ensure_complete_sentence(response)

            # Basic quality check
            if len(response) > 10 and response.lower() != message.lower():
                return response

        except Exception as e:
            print(f"Model generation error: {e}")

    # Minimal fallback - only when model completely fails
    return "I'm here to listen and support you. Can you tell me more about what you're experiencing right now? "

# Custom CSS for narrow layout
custom_css = """
.gradio-container {
    max-width: 1200px !important;
    margin: 0 auto !important;
    padding: 20px !important;
}
.block {
    max-width: 800px !important;
}
.chat-message {
    font-size: 14px !important;
}
.input-wrap {
    max-width: 100% !important;
}
#chatbot {
    height: 400px !important;
    max-height: 400px !important;
}
.message {
    max-width: 90% !important;
}
"""

status_message = "loaded trained model" if model_loaded else "Model not loaded - using fallback"

with gr.Blocks(
    css=custom_css,
    theme=gr.themes.Soft(),
    title="MindSpace"
) as demo:

    gr.Markdown(f"""
    # 🧠 MindSpace - Mental Health Assistant

    **Status:** {status_message}

    Your AI mental health companion trained on therapeutic conversations.
    Share your thoughts and feelings in a supportive environment.

    *This is an AI assistant, not a replacement for professional mental health care.*
    """)

    with gr.Column(scale=1):
        chatbot = gr.Chatbot(
            value=[],
            elem_id="chatbot",
            height=400,
            show_label=False,
            container=True,
            scale=1
        )

        with gr.Row():
            msg = gr.Textbox(
                placeholder="Share what's on your mind...",
                show_label=False,
                scale=4,
                container=False
            )
            submit_btn = gr.Button("Send", scale=1, variant="primary")

    # Example buttons in a compact layout
    with gr.Row():
        gr.Examples(
            examples=[
                "I'm feeling happy because I passed my interview",
                "I've been anxious about work lately",
                "I'm going through a difficult time",
                "I accomplished something important today"
            ],
            inputs=msg,
            label="Try these examples:"
        )

    def respond(message, chat_history):
        bot_message = therapy_response(message, chat_history)
        chat_history.append((message, bot_message))
        return "", chat_history

    # Event handlers
    msg.submit(respond, [msg, chatbot], [msg, chatbot])
    submit_btn.click(respond, [msg, chatbot], [msg, chatbot])

print(f"Status: {status_message}")
demo.launch(share=True)

Status: loaded trained model


/tmp/ipython-input-1851870758.py:159: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://68b6d554c1335d7ea4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
